In [9]:
import pandas as pd
import os
import numpy as np
from statsmodels.tsa.stattools import pacf, ccf
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor

import hydroeval as he
import scipy

import joblib

import matplotlib.pyplot as plt
import glob

from sklearn.metrics import mean_absolute_error, r2_score
from datetime import datetime

In [34]:
current_datetime = datetime.now().strftime("%Y%m%d_%H%M")
input_folder = "./data/recursive/run_gap_station_csv"
original_data_folder = "./data/recursive/best_13_clean_data_csv"
results_folder = os.path.join(input_folder, f"results_{current_datetime}")

os.makedirs(results_folder, exist_ok=True)

In [37]:
num_pacf_lags = 3  # Number of PACF lags to use
num_ccf_lags = 5  # Number of CCF lags to use

print_plot = True
modeltype = "rf"

In [38]:
files = glob.glob("./data/recursive/best_13_clean_data_csv/*.csv")
files

['./data/recursive/best_13_clean_data_csv/station_916.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_962.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_4620.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_958.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_1.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_986.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_1717.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_4649.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_924.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_1712.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_1715.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_927.0_cleaned.csv',
 './data/recursive/best_13_clean_data_csv/station_1718.0_cleaned.csv']

In [39]:
def plotTopCorrelations(pacf_values, ccf_vals, station_name, input_file_path):
    # Plot and save PACF
    plt.figure(figsize=(10, 5))
    plt.stem(range(len(pacf_values)), pacf_values, use_line_collection=True)
    plt.title(
        f"Partial Autocorrelation Function (PACF) - {station_name}\n{os.path.basename(input_file_path)}",
        fontsize=20,
    )
    plt.xlabel("Lag", fontsize=15)
    plt.ylabel("PACF", fontsize=15)
    pacf_plot_file_name = os.path.join(
        results_folder,
        f"acf_{os.path.splitext(os.path.basename(input_file_path))[0]}.png",
    )
    plt.savefig(pacf_plot_file_name, dpi=300)
    plt.close()

    # Plot and save CCF
    plt.figure(figsize=(10, 5))
    plt.stem(range(len(ccf_vals)), ccf_vals, use_line_collection=True)
    plt.title(
        f"Cross-Correlation Function (CCF) - {station_name}\n{os.path.basename(input_file_path)}",
        fontsize=20,
    )
    plt.xlabel("Lag", fontsize=15)
    plt.ylabel("CCF", fontsize=15)
    ccf_plot_file_name = os.path.join(
        results_folder,
        f"ccf_{os.path.splitext(os.path.basename(input_file_path))[0]}.png",
    )
    plt.savefig(ccf_plot_file_name, dpi=300)
    plt.close()

In [40]:
# Section3
# Function to display missing value, precipitation, and discharge plots.
def plots(
    df, col_dis, col_prec, station_name, input_file_path, results_folder, nan_mask
):
    nan_indices = nan_mask[nan_mask].index

    # Finding the start and end indices of consecutive NaN sequences
    consecutive_nans = []
    start_index = None

    for i in range(len(nan_indices)):
        if start_index is None:
            start_index = nan_indices[i]
        if i == len(nan_indices) - 1 or nan_indices[i + 1] != nan_indices[
            i
        ] + pd.Timedelta(days=1):
            consecutive_nans.append((start_index, nan_indices[i]))
            start_index = None

    # Plot and save Missing Values Chart
    df["missing_data"] = np.where(df[col_dis].isna(), 0, 1)
    plt.figure(figsize=(15, 6))
    plt.plot(
        df.index.values, df["missing_data"].values, label="Presence and absence of data"
    )
    plt.xlabel("Time", fontsize=15)
    plt.ylabel("Missing Data Indicator", fontsize=15)
    plt.yticks(np.arange(0, 1.1, step=1))
    plt.title(
        f"Missing Data over Time - {station_name}\n{os.path.basename(input_file_path)}",
        fontsize=20,
    )
    plt.legend(fontsize=15)

    # Save the plot as an image file in the results folder
    missing_values_plot_file_name = os.path.join(
        results_folder,
        f"miss_values_{os.path.splitext(os.path.basename(input_file_path))[0]}.png",
    )
    plt.savefig(missing_values_plot_file_name, dpi=300)
    # Close the plot to avoid displaying it
    plt.close()

    # Plot Streamflow Chart
    plt.figure(figsize=(20, 12))
    plt.plot(df.index.values, df[col_dis].values, label="Streamflow")
    plt.xlabel("Date", fontsize=15)
    plt.ylabel("Streamflow", fontsize=15)
    plt.title(
        f"Streamflow over Time - {station_name}\n{os.path.basename(input_file_path)}",
        fontsize=20,
    )
    plt.legend(fontsize=15)
    for start, end in consecutive_nans:
        plt.axvline(start, color="r", linestyle="--")
        plt.axvline(end, color="r", linestyle="--")

    # Save the streamflow plot as an image file in the results folder
    streamflow_plot_file_name = os.path.join(
        results_folder,
        f"flow_vs_time_{os.path.splitext(os.path.basename(input_file_path))[0]}.png",
    )
    plt.savefig(streamflow_plot_file_name, dpi=300)
    # Close the plot to avoid displaying it
    plt.close()

    # Plot and save Precipitation Chart
    plt.figure(figsize=(25, 12))
    plt.plot(df.index.values, df[col_prec].values, label="Precipitation")
    plt.xlabel("Date", fontsize=15)
    plt.ylabel("Precipitation", fontsize=15)
    plt.title(
        f"Precipitation over Time - {station_name}\n{os.path.basename(input_file_path)}",
        fontsize=20,
    )
    plt.legend(fontsize=15)

    # Save the precipitation plot as an image file in the results folder
    precipitation_plot_file_name = os.path.join(
        results_folder,
        f"precip_vs_time_{os.path.splitext(os.path.basename(input_file_path))[0]}.png",
    )
    plt.savefig(precipitation_plot_file_name, dpi=300)

    # Close the plot to avoid displaying it
    plt.close()

In [41]:
def getTopCorrelations(df, col_Dis, col_Prec, n_pacf, n_ccf):
    # Calculate PACF values for autocorrelation
    pacf_values = pacf(df[col_Dis], nlags=n_pacf)
    pacf_values = pd.Series(
        pacf_values[1:], index=np.arange(1, len(pacf_values))
    )  # Exclude lag 0

    # Filter PACF values with correlation coefficient > 0.5
    filtered_pacf = pacf_values[abs(pacf_values) > 0.5]

    # Select up to n_pacf lags
    top_pacf_lags = filtered_pacf.head(n_pacf).index.tolist()

    # Calculate CCF values for cross-correlation
    ccf_vals = ccf(df[col_Dis], df[col_Prec])
    ccf_vals = pd.Series(
        ccf_vals[1:], index=np.arange(1, len(ccf_vals))
    )  # Exclude lag 0

    # Filter CCF values with correlation coefficient > 0.5
    filtered_ccf = ccf_vals[abs(ccf_vals) > 0.5]

    # Select up to n_ccf lags
    top_ccf_lags = filtered_ccf.head(n_ccf).index.tolist()

    return top_pacf_lags, top_ccf_lags, pacf_values, ccf_vals

In [52]:
# Section2
def process_file(input_file_path, original_data_folder, print_plot=False):
    # Read and print CSV file
    data = pd.read_csv(input_file_path, parse_dates=["time"], index_col="time")

    # Extract station name from the input file name
    station_name = "_".join(
        os.path.basename(input_file_path).split("_")[:2]
    )  # Extract "station_X.0"

    # Detect gaps automatically
    nan_mask = data["obsdis"].isna()
    gap_start = nan_mask.idxmax()  # The first NaN value
    gap_end = nan_mask[::-1].idxmax()  # The last NaN value

    if print_plot:
        plots(
            data,
            "obsdis",
            "tp",
            station_name,
            input_file_path,
            results_folder,
            nan_mask,
        )

    # Section4
    data = data.asfreq("D")
    data = data[["tp", "obsdis"]]
    data_without_NaN = data.dropna()

    _, _, pacf_vals, ccf_vals = getTopCorrelations(
        data_without_NaN, "obsdis", "tp", num_pacf_lags, num_ccf_lags
    )

    if print_plot:
        plotTopCorrelations(pacf_vals, ccf_vals, station_name, input_file_path)

    # Section6
    def preprocess_data(df, Q_lags=[], P_lags=[]):
        def normalization(df):
            scaler_tp = StandardScaler()
            scaler_obsdis = StandardScaler()

            scaled_tp = scaler_tp.fit_transform(df[["tp"]])
            scaled_obsdis = scaler_obsdis.fit_transform(df[["obsdis"]])

            scaled_df = pd.DataFrame(
                {"tp": scaled_tp.flatten(), "obsdis": scaled_obsdis.flatten()},
                index=df.index,
            )

            return scaled_df, scaler_tp, scaler_obsdis

        def create_features(df):
            df["month"] = df.index.month
            df["quarter"] = df.index.quarter
            return df

        def create_Q_lag_features(df, lags):
            for lag in lags:
                df[f"Q_lag_time_{lag}"] = df["obsdis"].shift(periods=lag)
            return df

        def create_P_lag_features(df, lags):
            for lag in lags:
                df[f"P_lag_time_{lag}"] = df["tp"].shift(periods=lag)
            return df

        def create_dummy_variables(df):
            df = pd.get_dummies(df, columns=["month", "quarter"], dtype=float)
            return df

        df, scaler_tp, scaler_obsdis = normalization(df)
        df = create_features(df)

        if Q_lags:
            df = create_Q_lag_features(df, Q_lags)

        if P_lags:
            df = create_P_lag_features(df, P_lags)

        df = create_dummy_variables(df)

        return df, scaler_tp, scaler_obsdis

    # Section7
    # Q_lags = top_autocorrelations
    # P_lags = top_crosscorrelations
    Q_lags = [1, 2, 3]
    P_lags = [1, 2, 3, 4, 5]
    training, scaler_tp, scaler_obsdis = preprocess_data(
        data_without_NaN, Q_lags=Q_lags, P_lags=P_lags
    )
    training = training.dropna()

    # Section8
    data_with_gaps, scaler_tp, scaler_obsdis = preprocess_data(
        data, Q_lags=Q_lags, P_lags=P_lags
    )
    gaps = data_with_gaps.loc[gap_start:gap_end]
    gaps = gaps.drop(["tp", "obsdis"], axis=1)

    # Section9
    def train_model(x_train, y_train, model_type):
        # Train a model based on the specified type.
        if model_type == "rf":
            model = RandomForestRegressor(random_state=42)
        elif model_type == "xgb":
            model = xgb.XGBRegressor(
                objective="reg:squarederror", eval_metric="rmse", random_state=42
            )
        elif model_type == "lgb":
            model = lgb.LGBMRegressor(
                objective="regression", metric="rmse", random_state=42
            )
        else:
            raise ValueError("Unsupported model type. Choose from 'rf', 'xgb', 'lgb'.")

        model.fit(x_train, y_train)
        return model

    # Training
    steps = 1
    data_train = training[:-steps]
    data_test = training[-steps:]
    x_train = data_train.drop(columns=["obsdis", "tp"])
    y_train = data_train["obsdis"]

    # TODO: this guys are unused
    x_test = data_test.drop(columns=["obsdis", "tp"])
    y_test = data_test["obsdis"]

    model = train_model(x_train, y_train, model_type=modeltype)

    # Section10
    def update_features(previous_input, new_prediction, num_lags):
        updated_input = previous_input.copy()
        # Update Q lag features dynamically based on the number of lags
        for i in range(num_lags - 1, 0, -1):
            updated_input[i] = updated_input[i - 1]  # Shift lag values
        updated_input[0] = new_prediction  # Set the new prediction as the first lag
        return updated_input

    def recursive_forecasting(initial_input, model, n_steps, gaps_df):
        predictions = []
        current_input = initial_input.flatten()

        for step in range(n_steps):
            # Predict the next value
            next_prediction = model.predict(current_input.reshape(1, -1))[0]
            predictions.append(next_prediction)

            # Update the input features for the next step
            if step + 1 < len(gaps_df):
                # Update Q lag values
                updated_q_lags = update_features(
                    current_input[:num_pacf_lags], next_prediction, num_pacf_lags
                )
                # Retain P lag values from the corresponding row in gaps_df
                p_lags = gaps_df.iloc[step + 1, num_pacf_lags:].values
                # Concatenate updated Q lags and unchanged P lags
                current_input = np.concatenate((updated_q_lags, p_lags))
        return predictions

    initial_test_input = gaps.iloc[0].values.reshape(1, -1)
    n_steps_ahead = len(gaps)
    multi_step_predictions = recursive_forecasting(
        initial_test_input, model, n_steps_ahead, gaps
    )
    gaps["obsdis"] = multi_step_predictions

    # Section11
    scaler_obsdis = StandardScaler()

    # TODO: scaled_obsdis is not used
    scaled_obsdis = scaler_obsdis.fit_transform(data[["obsdis"]])
    multi_step_predictions_denorm = scaler_obsdis.inverse_transform(
        np.array(multi_step_predictions).reshape(-1, 1)
    )

    # Section12
    ungapped_file_name = f"{station_name}_cleaned.csv"
    original_file_path = os.path.join(original_data_folder, ungapped_file_name)
    val = pd.read_csv(original_file_path, index_col=0, parse_dates=True)
    val = val["obsdis"].loc[gap_start:gap_end]
    val = pd.DataFrame(val)

    # Section13
    dates = pd.date_range(start=gap_start, periods=len(val), freq="D")
    val = pd.DataFrame(val, index=dates)
    multi_step_predictions_denorm = pd.DataFrame(
        multi_step_predictions_denorm, columns=["predicted"], index=dates
    )
    combined_df = pd.concat([val, multi_step_predictions_denorm], axis=1)

    # Section14
    def index_of_agreement(obs, cal):
        mean_obs = np.mean(obs)
        ioa = 1 - (np.sum((obs - cal) ** 2)) / (
            np.sum((np.abs(cal - mean_obs) + np.abs(obs - mean_obs)) ** 2)
        )
        return ioa

    kge_values = he.evaluator(he.kge, combined_df["predicted"], combined_df["obsdis"])
    kge_overall = float(kge_values[0])
    kge_correlation = float(kge_values[1])
    kge_bias = float(kge_values[2])
    kge_variability = float(kge_values[3])

    # Check if there are at least two data points for Pearson correlation
    if len(combined_df["obsdis"]) >= 2 and len(combined_df["predicted"]) >= 2:
        pearson_corr = scipy.stats.pearsonr(
            combined_df["obsdis"], combined_df["predicted"]
        )[0]
    else:
        pearson_corr = None

    metrics = {
        "Q_lags": Q_lags,  # Add Q_lags array to metrics
        "Q_lags_Coefficients": [
            round(pacf_vals[lag], 3) for lag in Q_lags
        ],  # Add Q_lags correlation coefficients rounded to 3 decimals
        "P_lags": P_lags,  # Add P_lags array to metrics
        "P_lags_Coefficients": [
            round(ccf_vals[lag], 3) for lag in P_lags
        ],  # Add P_lags correlation coefficients rounded to 3 decimals
        "Input File": input_file_path,
        "R2_score": r2_score(combined_df["obsdis"], combined_df["predicted"]),
        "RMSE": np.sqrt(
            np.mean((combined_df["predicted"] - combined_df["obsdis"]) ** 2)
        ),
        "Mean Bias Error": np.mean(combined_df["predicted"] - combined_df["obsdis"]),
        "MAE": mean_absolute_error(combined_df["obsdis"], combined_df["predicted"]),
        "Nash-Sutcliffe": float(
            he.evaluator(he.nse, combined_df["predicted"], combined_df["obsdis"])[0]
        ),
        "Index of Agreement": index_of_agreement(
            combined_df["obsdis"], combined_df["predicted"]
        ),
        "Correlation Coefficient": pearson_corr,
        "KGE Overall": kge_overall,
        "KGE Correlation": kge_correlation,
        "KGE Bias": kge_bias,
        "KGE Variability": kge_variability,
    }

    return model, combined_df, metrics, station_name

In [53]:
files[0]

'./data/recursive/best_13_clean_data_csv/station_916.0_cleaned.csv'

In [54]:
print_plot

True

In [55]:
station_file = files[0]

model, combined_df, metrics, station_name = process_file(
    station_file, original_data_folder, print_plot=print_plot
)

/var/folders/d6/1xmf_wcd7f3bkb179x6rc16r0000gn/T/ipykernel_34581/4180314264.py:4: MatplotlibDeprecationWarning: The 'use_line_collection' parameter of stem() was deprecated in Matplotlib 3.6 and will be removed two minor releases later. If any parameter follows 'use_line_collection', they should be passed as keyword, not positionally.
  plt.stem(range(len(pacf_values)), pacf_values, use_line_collection=True)
/var/folders/d6/1xmf_wcd7f3bkb179x6rc16r0000gn/T/ipykernel_34581/4180314264.py:14: MatplotlibDeprecationWarning: The 'use_line_collection' parameter of stem() was deprecated in Matplotlib 3.6 and will be removed two minor releases later. If any parameter follows 'use_line_collection', they should be passed as keyword, not positionally.
  plt.stem(range(len(ccf_vals)), ccf_vals, use_line_collection=True)
/Users/konstantinosperifanos/code/venvs/py3/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted 

In [56]:
model

RandomForestRegressor(random_state=42)

In [ ]:
joblib.dump(f"./models/{station_name}_{modeltype}.pkl")